In [1]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [2]:
dim = 2
mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
#mesh_file = "/home/ayoub.hamou/mesh/square.msh"
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 105826
Number of Vertices: 53314
Local domain contruction ...


In [3]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,

])

args = [
  ne.cell,
  ne.ghost,
  ne.halo,
  ne.gradcellx,
  ne.gradcelly,
  ne.gradcellz,
  ne.psi,
  ne.domain.faces.cellid,
  ne.domain.cells.faceid,
  ne.domain.faces.name,
  ne.domain.faces.halofid,
  ne.domain.cells.center,
  ne.domain.faces.center,
]



---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [4]:
from manapy.ast.functions2d import barthlimiter_2d as cpu_function
from manapy.cuda.manapy.ast.cuda_functions2d import get_kernel_barthlimiter_2d as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile barthlimiter_2d to cpu => signature=(float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:,:], int32[:], int32[:], float32[:,:], float32[:,:])
compile kernel_barthlimiter_2d to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:,:], int32[:], int32[:], float32[:,:], float32[:,:])


In [5]:
numba_fun(*args)
%timeit numba_fun(*args)

8.79 ms ± 503 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_barthlimiter_2d.<locals>.kernel_barthlimiter_2d at 0x7154ce161820>) is called
VarClass<float32, (105826,)>
VarClass<float32, (159139,)>
VarClass<float32, (0,)>
VarClass<float32, (105826,)>
VarClass<float32, (105826,)>
VarClass<float32, (105826,)>
VarClass<float32, (105826,)>
VarClass<int32, (159139, 2)>
VarClass<int32, (105826, 4)>
VarClass<int32, (159139,)>
VarClass<int32, (159139,)>
VarClass<float32, (105826, 3)>
VarClass<float32, (159139, 3)>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


1.7 ms ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
def verify(a, decimal, plot):
  b = VarClass.to_device(a)
  if plot == True:
    plt.plot(a, label="cpu")
    plt.plot(b, label="gpu")
    plt.legend()
    plt.show()
  np.testing.assert_almost_equal(a, b, decimal=decimal)


for i, arg in enumerate(args):
  print(f"{i + 1} => test arg {arg}")
  verify(arg, decimal=2, plot=False)

1 => test arg [1. 1. 1. ... 1. 1. 1.]
2 => test arg [0. 0. 0. ... 0. 0. 0.]
3 => test arg []
4 => test arg [0. 0. 0. ... 0. 0. 0.]
5 => test arg [0. 0. 0. ... 0. 0. 0.]
6 => test arg [0. 0. 0. ... 0. 0. 0.]
7 => test arg [1. 1. 1. ... 1. 1. 1.]
8 => test arg [[ 52584     -1]
 [ 53956     -1]
 [ 14290     -1]
 ...
 [  6880  15456]
 [ 65167  69113]
 [104411 105736]]
9 => test arg [[  4841  83565  83566      3]
 [   923  14658  14660      3]
 [   924  14654  14656      3]
 ...
 [157566 157575 157571      3]
 [157555 157560 157564      3]
 [157544 157553 157549      3]]
10 => test arg [3 3 3 ... 0 0 0]
11 => test arg [0 0 0 ... 0 0 0]
12 => test arg [[0.8517712  0.6815817  0.        ]
 [0.9459748  0.738177   0.        ]
 [0.7377134  0.05404253 0.        ]
 ...
 [0.9256547  0.23342907 0.        ]
 [0.07439401 0.7664809  0.        ]
 [0.7664369  0.925623   0.        ]]
13 => test arg [[0.0025     1.         0.        ]
 [0.0075     1.         0.        ]
 [0.0125     1.         0.        ]
 